In [1]:
import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
from llm.prompt_loader import PromptLoader
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from llm.ml_tools import svd_embed_score
from sklearn.decomposition import PCA
from metric_utils import get_measures, print_measures
from linear_probe import get_linear_acc
import os
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve

In [2]:
data_reader=ReadData('/home/linear_probing/output/pope')
data=data_reader.read_all()
data[0]

{'img_path': '/root/autodl-fs/coco_images/train/COCO_train2014_000000580392.jpg',
 'question': 'Is there a person in the image?\nAnswer the question using a single word or phrase.',
 'label': 1,
 'question_id': 5429,
 'category': 'popular',
 'most_likely': {'embedding': array([[[-1.8997192e-03,  1.8188477e-02, -4.3640137e-03, ...,
            9.0942383e-03, -9.1171265e-04,  1.2084961e-02]],
  
         [[-4.7264099e-03,  3.2684326e-02, -1.2527466e-02, ...,
            1.9760132e-02, -2.6321411e-03,  1.7044067e-02]],
  
         [[-1.6998291e-02,  6.8298340e-02, -3.8452148e-02, ...,
            5.6579590e-02,  1.0887146e-02,  3.4545898e-02]],
  
         ...,
  
         [[-1.3359375e+00,  1.0048828e+00, -6.2011719e-01, ...,
            4.6752930e-01,  1.1005859e+00, -8.0664062e-01]],
  
         [[-9.2675781e-01,  1.1953125e+00, -8.7158203e-02, ...,
            4.9487305e-01,  1.2353516e+00, -1.5087891e+00]],
  
         [[-6.4111328e-01, -2.6318359e-01, -2.4490356e-02, ...,
          

In [3]:
df=pd.DataFrame(data)

In [4]:
df.head()

,img_path,question,label,question_id,category,most_likely
0,/root/autodl-fs/coco_images/train/COCO_train20...,Is there a person in the image?\nAnswer the qu...,1,5429,popular,{'embedding': [[[-0.00189972 0.01818848 -0.00...
1,/root/autodl-fs/coco_images/train/COCO_train20...,Is there a snowboard in the image?\nAnswer the...,1,1619,adversarial,{'embedding': [[[-0.00189972 0.01818848 -0.00...
2,/root/autodl-fs/coco_images/train/COCO_train20...,Is there a car in the image?\nAnswer the quest...,0,846,popular,{'embedding': [[[-0.00189972 0.01818848 -0.00...
3,/root/autodl-fs/coco_images/train/COCO_train20...,Is there a skateboard in the image?\nAnswer th...,0,1906,random,{'embedding': [[[-0.00189972 0.01818848 -0.00...
4,/root/autodl-fs/coco_images/train/COCO_train20...,Is there a microwave in the image?\nAnswer the...,0,4598,random,{'embedding': [[[-0.00189972 0.01818848 -0.00...


In [5]:
gt_label=df['label'].to_numpy()

In [6]:
length=len(data)
wild_ratio=0.9

all_indices = np.random.permutation(length)
train_val_idxs = all_indices[:int(
    wild_ratio * length)]  # trainset and validation test index

test_idxs=all_indices[int(
    wild_ratio * length):]  #test index

validset_len = 300
# exclude validation samples.
train_idxs = train_val_idxs[:len(
    train_val_idxs) - validset_len]  # trainset index
val_idxs = train_val_idxs[len(
    train_val_idxs) - validset_len:]  # validation index
gt_label_test = []
gt_label_wild = []
gt_label_val = []

for i in range(length):
    if i not in train_val_idxs:
        gt_label_test.extend(gt_label[i: i+1])
    elif i in train_idxs:
        gt_label_wild.extend(gt_label[i: i+1])
    else:
        gt_label_val.extend(gt_label[i: i+1])

'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
gt_label_test = np.asarray(gt_label_test)
gt_label_wild = np.asarray(gt_label_wild)
gt_label_val = np.asarray(gt_label_val)

print(f'trainset length:{len(gt_label_wild)}')
print(f'validset length:{len(gt_label_val)}')
print(f'testset length:{len(gt_label_test)}')
print(f'trainset postive: {sum(gt_label_wild==1)} negative: {sum(gt_label_wild==0)}')
print(
    f'testset postive: {sum(gt_label_test==1)} negative: {sum(gt_label_test==0)}')

trainset length:600
validset length:300
testset length:100
trainset postive: 311 negative: 289
testset postive: 45 negative: 55


In [7]:
embed_generated = df['most_likely'].apply(lambda x: np.squeeze(x['embedding'])).to_list()
embed_generated =np.stack(embed_generated)
embed_generated.shape

(1000, 33, 4096)

In [8]:
feat_indices_wild = []
feat_indices_eval = []

for i in range(length):
    if i in train_idxs:
        feat_indices_wild.extend(np.arange(i, i+1).tolist())
    elif i in val_idxs:
        feat_indices_eval.extend(np.arange(i, i + 1).tolist())
embed_generated_wild = embed_generated[feat_indices_wild][:, 1:, :]
embed_generated_eval = embed_generated[feat_indices_eval][:, 1:, :]

In [ ]:
# k_span is the max k value
# graid search for best hyper-parameters on validation set, and store them into returned_results
weighted_svd=True
returned_results = svd_embed_score(embed_generated_eval, gt_label_val,
                                   begin_k=1, k_span=11, mean=0, svd=0, weight=weighted_svd)

pca_model = PCA(n_components=returned_results['k'], whiten=False).fit(
    embed_generated_wild[:, returned_results['best_layer'], :])
projection = pca_model.components_.T
if weighted_svd:
    projection = pca_model.singular_values_ * projection
scores = np.mean(np.matmul(
    embed_generated_wild[:, returned_results['best_layer'], :], projection), -1, keepdims=True)
assert scores.shape[1] == 1
best_scores = np.sqrt(np.sum(np.square(scores), axis=1)
                      ) * returned_results['best_sign']

'''get score for direct projection'''
# direct projection

feat_indices_test = []
for i in range(length):
    if i not in train_val_idxs:
        feat_indices_test.extend(np.arange(1 * i, 1 * i + 1).tolist())

embed_generated_test = embed_generated[feat_indices_test][:, 1:, :]

test_scores = np.mean(np.matmul(embed_generated_test[:, returned_results['best_layer'], :],
                                projection), -1, keepdims=True)

assert test_scores.shape[1] == 1
test_scores = np.sqrt(np.sum(np.square(test_scores), axis=1))

measures = get_measures(returned_results['best_sign'] * test_scores[gt_label_test == 1],
                        returned_results['best_sign'] * test_scores[gt_label_test == 0], plot=False)
print_measures(measures[0], measures[1], measures[2], 'direct-projection')

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:00<00:06,  1.39it/s]

k:  1 AUROC:83.38818715531043 layer:  3 mean:  0 svd:  0


 20%|██        | 2/10 [00:01<00:05,  1.38it/s]

k:  2 AUROC:84.51343177370575 layer:  30 mean:  0 svd:  0


...

k:  8 AUROC:83.39708236968511 layer:  28 mean:  0 svd:  0


 90%|█████████ | 9/10 [00:06<00:00,  1.27it/s]

k:  9 AUROC:83.12133072407045 layer:  28 mean:  0 svd:  0


100%|██████████| 10/10 [00:07<00:00,  1.31it/s]

k:  10 AUROC:83.33481586906244 layer:  28 mean:  0 svd:  0
				direct-projection
  FPR95 AUROC AUPR
& 41.82 & 92.32 & 90.99


In [10]:
print(f'best_score distributes in range: {best_scores.max()}-{best_scores.min()}')

best_score distributes in range: 4473.31689453125-1.3375941514968872


In [11]:
feat_indices_test = []
for i in range(length):
    if i not in train_val_idxs:
        feat_indices_test.extend(np.arange(1 * i, 1 * i + 1).tolist())

embed_generated_test = embed_generated[feat_indices_test][:, 1:, :]


In [12]:
thresholds = np.linspace(0, 1, num=40)[1:-1]

# graid search
auroc_over_thres = []
for thres_wild in tqdm(thresholds):
    best_auroc = 0
    best_accuracy=0
    for layer in range(len(embed_generated_wild[0])):
        thres_wild_score = np.sort(best_scores)[
            int(len(best_scores) * thres_wild)]
        true_wild = embed_generated_wild[:, layer,:][best_scores > thres_wild_score]
        false_wild = embed_generated_wild[:, layer,:][best_scores <= thres_wild_score]

        embed_train = np.concatenate([true_wild, false_wild], 0) # true label: 1; false label:0
        label_train = np.concatenate([np.ones(len(true_wild)),
                                      np.zeros(len(false_wild))], 0)

        # gt training, saplma
        # embed_train = embed_generated_wild[:,layer,:]
        # label_train = gt_label_wild
        # gt training, saplma
        print(f'postive: {sum(label_train==1)} negative: {sum(label_train==0)}')

        best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            embed_train,
            label_train,
            embed_train,
            label_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

        clf.eval()
        output = clf(torch.from_numpy(
            embed_generated_test[:, layer, :]).to(torch.float32).cuda())
        pca_wild_score_binary_cls = torch.sigmoid(output)

        pca_wild_score_binary_cls = pca_wild_score_binary_cls.cpu().data.numpy()

        #pred_labels=(pca_wild_score_binary_cls >= 0.5).astype(int)
        
        #pred_acc=accuracy_score(gt_label_test,pred_labels)

        if np.isnan(pca_wild_score_binary_cls).sum() > 0:
            breakpoint()
        measures = get_measures(pca_wild_score_binary_cls[gt_label_test == 1],
                                pca_wild_score_binary_cls[gt_label_test == 0], plot=False)
        
        #auroc=roc_auc_score(gt_label_test,pca_wild_score_binary_cls)
        #if auroc> best_auroc:
        #    best_auroc=auroc
        #    best_accuracy=pred_acc
        #    best_layer=layer

        #if pred_acc> best_accuracy:
        #    best_accuracy=pred_acc
        
        
        if measures[0] > best_auroc:
            best_auroc = measures[0]
            best_result = [100 * measures[0]]

            fpr, tpr, thresholds = roc_curve(gt_label_test,pca_wild_score_binary_cls)

            # Calculate Youden's J statistic
            youdens_j = tpr - fpr
            # Find the index of the maximum J statistic 
            best_index = np.argmax(youdens_j) 
            best_threshold = thresholds[best_index]

            pred_labels=(pca_wild_score_binary_cls >= best_threshold).astype(int)
            best_accuracy=accuracy_score(gt_label_test,pred_labels)

            best_layer = layer


    print('thres: ', np.sort(best_scores)[
            int(len(best_scores) * thres_wild)], 'best result: ',
          best_result, 'best_layer: ', best_layer,'best_accuracy',best_accuracy,'linear best_threshold',best_threshold)

  0%|          | 0/38 [00:00<?, ?it/s]

postive: 584 negative: 16
postive: 584 negative: 16
postive: 584 negative: 16
postive: 584 negative: 16
postive: 584 negative: 16
postive: 584 negative: 16
postive: 584 negative: 16
postive: 584 negative: 16
postive: 584 negative: 16
postive: 584 negative: 16
postive: 584 negative: 16
postive: 584 negative: 16
postive: 584 negative: 16
postive: 584 negative: 16
postive: 584 negative: 16
postive: 584 negative: 16
postive: 584 negative: 16
postive: 584 negative: 16
postive: 584 negative: 16
postive: 584 negative: 16
postive: 584 negative: 16
postive: 584 negative: 16
postive: 584 negative: 16
postive: 584 negative: 16
postive: 584 negative: 16
postive: 584 negative: 16
postive: 584 negative: 16
postive: 584 negative: 16
postive: 584 negative: 16
postive: 584 negative: 16
postive: 584 negative: 16
postive: 584 negative: 16


  3%|▎         | 1/38 [00:23<14:41, 23.83s/it]

thres:  45.962856 best result:  [np.float64(93.61616161616162)] best_layer:  26 best_accuracy 0.89 linear best_threshold 0.9655429
postive: 569 negative: 31
postive: 569 negative: 31
postive: 569 negative: 31
postive: 569 negative: 31
postive: 569 negative: 31
postive: 569 negative: 31
postive: 569 negative: 31
postive: 569 negative: 31
postive: 569 negative: 31
postive: 569 negative: 31
postive: 569 negative: 31
postive: 569 negative: 31
postive: 569 negative: 31
postive: 569 negative: 31
postive: 569 negative: 31
postive: 569 negative: 31
postive: 569 negative: 31
postive: 569 negative: 31
postive: 569 negative: 31
postive: 569 negative: 31
postive: 569 negative: 31
postive: 569 negative: 31
postive: 569 negative: 31
postive: 569 negative: 31
postive: 569 negative: 31
postive: 569 negative: 31
postive: 569 negative: 31
postive: 569 negative: 31
postive: 569 negative: 31
postive: 569 negative: 31
postive: 569 negative: 31
postive: 569 negative: 31


  5%|▌         | 2/38 [00:44<13:20, 22.24s/it]

...

 87%|████████▋ | 33/38 [12:12<01:48, 21.79s/it]

thres:  3978.8293 best result:  [np.float64(94.46464646464646)] best_layer:  18 best_accuracy 0.9 linear best_threshold 2.5638804e-09
postive: 76 negative: 524
postive: 76 negative: 524
postive: 76 negative: 524
postive: 76 negative: 524
postive: 76 negative: 524
postive: 76 negative: 524
postive: 76 negative: 524
postive: 76 negative: 524
postive: 76 negative: 524
postive: 76 negative: 524
postive: 76 negative: 524
postive: 76 negative: 524


 87%|████████▋ | 33/38 [12:19<01:52, 22.42s/it]

postive: 76 negative: 524


KeyboardInterrupt: 